In [13]:
import pandas as pd
import numpy as np
import yfinance as yf
import sqlite3
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import re

In [14]:
stocks={'Google' : 'GOOGL',
'Amazon' : 'AMZN',
'Microsoft' : 'MSFT',
'Tesla' : 'TSLA',
'Facebook' : 'META'}

In [15]:
# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('stock_data.db')
cursor = conn.cursor()


In [16]:
for name, ticker in stocks.items():

    # Get stock data from Yahoo Finance
    try:
        stock = yf.Ticker(ticker).history(period='10y')
        if stock.empty:
            raise ValueError(f"No data available for {name} ({ticker})")
    except Exception as e:
        print(f"Error fetching {name} ({ticker}): {e}")
        continue
    print(f'{name} ({ticker})')

    #convert the data to a pandas DataFrame
    df = pd.DataFrame(stock, columns=['Open', 'High', 'Low', 'Close', 'Volume'])
    df.index = df.index.date
    print(df.info())
    if not df.empty:
        print(df.sample(5))
    else:
        print(f"No data available for {name}")
    print('='*100)

    # Save the stock data to a SQLite database
    sanitized_name = re.sub(r'[^a-zA-Z0-9]', '_', name)
    cursor.execute(f'''
    CREATE TABLE IF NOT EXISTS stock_prices_{sanitized_name} (
        date TEXT PRIMARY KEY,
        open REAL,
        high REAL,
        low REAL,
        close REAL,
        volume BIGINT
        )
    ''')
    df.to_sql(f'stock_prices_{sanitized_name}', conn, if_exists='replace', index_label='date')
conn.commit()
conn.close()


Google (GOOGL)
<class 'pandas.core.frame.DataFrame'>
Index: 2515 entries, 2015-02-04 to 2025-02-03
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    2515 non-null   float64
 1   High    2515 non-null   float64
 2   Low     2515 non-null   float64
 3   Close   2515 non-null   float64
 4   Volume  2515 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 117.9+ KB
None
                  Open        High         Low       Close    Volume
2021-09-28  138.154179  138.594082  134.871082  135.339386  45772000
2017-03-20   43.316970   43.359814   43.077341   43.238754  30844000
2024-06-20  174.937110  176.852369  174.558058  175.864822  20160100
2019-07-16   57.129397   57.719759   57.129397   57.464684  25352000
2020-02-14   75.452976   75.727980   75.005592   75.662216  23198000
Amazon (AMZN)
<class 'pandas.core.frame.DataFrame'>
Index: 2515 entries, 2015-02-04 to 2025-02-03
Data columns (total 5 columns):
 #   Col

In [17]:
# Connect to SQLite database
conn = sqlite3.connect('stock_data.db')
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print(tables)

[('stock_prices_Google',), ('stock_prices_Amazon',), ('stock_prices_Microsoft',), ('stock_prices_Tesla',), ('stock_prices_Facebook',)]


In [18]:
# Connect to SQLite database
conn = sqlite3.connect('stock_data.db')
cursor = conn.cursor()

# Get available tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

for table in tables:
    table = table[0]  # Extract table name
    print(f"Processing: {table}")
    
    # Load data from SQLite
    data = pd.read_sql(f'SELECT * FROM "{table}"', conn, index_col='date', parse_dates=['date'])

    # Calculate moving averages
    data['Volume_30_MA'] = data['Volume'].rolling(window=30).mean()
    data['90_day_MA'] = data['Close'].rolling(window=90).mean()
    data['180_day_MA'] = data['Close'].rolling(window=180).mean()

    # === 📈 STOCK PRICE CHART ===
    fig_price = go.Figure()

    # Stock Price & Moving Averages
    fig_price.add_trace(go.Scatter(x=data.index, y=data['Close'], mode='lines', name='Stock Price'))
    fig_price.add_trace(go.Scatter(x=data.index, y=data['90_day_MA'], mode='lines', name='90-day MA', line=dict(dash='dash')))
    fig_price.add_trace(go.Scatter(x=data.index, y=data['180_day_MA'], mode='lines', name='180-day MA', line=dict(dash='dot')))

    # Layout update
    fig_price.update_layout(
        title=f'Stock Price & Moving Averages for {table}',
        xaxis_title='Date',
        yaxis_title='Price',
        legend=dict(x=0, y=1),
        hovermode='x'
    )

    fig_price.show()

    # === 📊 VOLUME MOVING AVERAGE CHART ===
    fig_volume = go.Figure()

    fig_volume.add_trace(go.Bar(x=data.index, y=data['Volume'], name='Daily Volume'))
    fig_volume.add_trace(go.Scatter(x=data.index, y=data['Volume_30_MA'], mode='lines', name='Volume 30-day MA', line=dict(color='red')))

    # Layout update
    fig_volume.update_layout(
        title=f'Trading Volume & 30-day MA for {table}',
        xaxis_title='Date',
        yaxis_title='Volume',
        legend=dict(x=0, y=1),
        hovermode='x'
    )

    fig_volume.show()

# Close database connection
conn.close()


Processing: stock_prices_Google


Processing: stock_prices_Amazon


Processing: stock_prices_Microsoft


Processing: stock_prices_Tesla


Processing: stock_prices_Facebook
